# Run Lora Trainer XL Colab on Modal Jupyter Notebook

This notebook will set up an environment that can run 
__[XL Lora Trainer by Hollowstrawberry](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb)__


## 1. Setup Environment:
Run each cells below in order from top to bottom

1.1. The name of your project:

In [2]:
project_name = "test"

1.2. Prepare things

In [3]:
!apt -y update -qq > /dev/null 2>&1
!apt -y upgrade -qq > /dev/null 2>&1
!apt install -y aria2 -qq > /dev/null 2>&1
!curl https://rclone.org/install.sh | bash > /dev/null 2>&1
import os
root_dir = "/content"
trainer_dir = os.path.join(root_dir, "trainer")
drive_dir = f"{root_dir}/drive"
MyDrive_dir = f"{drive_dir}/MyDrive"
Lora_dir = f"{MyDrive_dir}/Loras"
project_dir = f"{Lora_dir}/{project_name}"
dataset_dir = f"{project_dir}/dataset"
!mkdir -p {root_dir}
!mkdir -p {drive_dir}
!mkdir -p {MyDrive_dir}
!mkdir -p {Lora_dir}
!mkdir -p {project_dir}
!mkdir -p {dataset_dir}
!mkdir /root/.config/
!mkdir /root/.config/rclone/
!git clone --recurse-submodules https://github.com/derrian-distro/LoRA_Easy_Training_scripts_Backend.git {trainer_dir} > /dev/null 2>&1
sd_scripts_dir = f"{trainer_dir}/sd_scripts"
requirements_path = f"{trainer_dir}/requirements.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4734  100  4734    0     0  13366      0 --:--:-- --:--:-- --:--:-- 13410
curl: (23) Failure writing output to destination
Cloning into '/content/trainer'...
remote: Enumerating objects: 388, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 388 (delta 181), reused 146 (delta 146), pack-reused 172 (from 2)
Receiving objects: 100% (388/388), 7.80 MiB | 26.10 MiB/s, done.
Resolving deltas: 100% (222/222), done.
Submodule 'LyCORIS' (https://github.com/67372a/LyCORIS) registered for path 'lycoris'
Submodule 'sd_scripts' (https://github.com/kohya-ss/sd-scripts) registered for path 'sd_scripts'
Cloning into '/content/trainer/lycoris'...
remote: Enumerating objects: 5158, done.        
remote: Counting objects: 100% (582/582), done.        
remote: Compressing object

1.3. Setup venv and install requires libraries

In [ ]:
os.chdir(sd_scripts_dir)
!python -m venv venv
!. venv/bin/activate && pip install --upgrade pip > /dev/null 2>&1
!. venv/bin/activate && pip install -r {requirements_path} > /dev/null 2>&1
!. venv/bin/activate && pip install accelerate transformers diffusers safetensors torch einops torchvision opencv-python-headless imagesize voluptuous bitsandbytes > /dev/null 2>&1 

## 2. Upload dataset

### __Instead of storing your Lora project files in Google Drive like when running in Google Colab, everything is stored locally in the Modal runtime environment. Therefore, you should not stop the Kernel without saving your data to your computer.__
### __You can browse the files in the "Files" tab.__

#### Location: /content/drive/MyDrive/Loras/{your-project-name}/dataset

## 4. Run Trainer

#### Set up all the parameters in advance on the Google Colab interface for easier and more intuitive operation, then copy the source code here and run it.

#### The Folder Structure must be "Organize by project".



#### Before run:
Replace 
```python 
    if not get_ipython().__dict__['user_ns']['_exit_code']:
```
with 
```python
    if not get_ipython().user_ns.get('_exit_code'):
```
to get rid of "KeyError"

You might also consider set `LOWRAM` to `False` since Modal's runtime environment has a lot more RAM than Google Colab's.

## 5. When Done:
### __Trained Lora files are stored locally in Modal's runtime instead of Google Drive. You should download it to your computer before terminate the Kernal because it will clear all your data.__
### Location: /content/drive/MyDrive/Loras/{your-project-name}/output
### Browse it on the "Files" tab.

## 5.1. Alternative: Using rclone

If you don't have time to just sit on the computer waiting for it to complete, you can use rclone to copy the trained Loras to your Google Drive

__If you use it, you should do steps 5.1.1 and 5.1.2 first, then run step 4: Run Trainer. Once the trainer is running, run step 5.1.3. This ensures that when you are not near the computer, when step 4 the trainer has finished training, your Lora will be copied to your Cloud by rclone immediately after.__

### 5.1.1. Upload `rclone.conf` file to '/root/.config/rclone'


### 5.1.2. Check if the 'rclone.conf' is loaded perfectly. If yes, it will list all Clouds that you have in your rclone.conf file.
#### You fill in the name of the Cloud you want rclone to copy your Lora to.

In [ ]:
!rclone listremotes

In [ ]:
cloud = ""

### 5.1.3. Run rclone 

In [ ]:
from_dir = f"{project_dir}/output"
to_dir = f"{cloud}:/Loras/{project_name}/output"
!rclone copy --multi-thread-streams=16 --update "$from_dir" "$to_dir" --ignore-existing --checksum --transfers 16 -v --stats 10s